In [1]:
# reading label data and filename from ./genres/input.mf
import pandas as pd
path = './genres/'
df = pd.read_csv(path + 'input.mf', delimiter ='\t', header=None, names=['fullpath', 'style'])
df['filename'] = df[:]['fullpath'].apply(lambda x: path+'/'.join(x.split('/')[-2:]))
df[:5]

,fullpath,style,filename
0,/Users/sness/mirex2008/genres/blues/blues.0000...,blues,./genres/blues/blues.00000.wav
1,/Users/sness/mirex2008/genres/blues/blues.0000...,blues,./genres/blues/blues.00001.wav
2,/Users/sness/mirex2008/genres/blues/blues.0000...,blues,./genres/blues/blues.00002.wav
3,/Users/sness/mirex2008/genres/blues/blues.0000...,blues,./genres/blues/blues.00003.wav
4,/Users/sness/mirex2008/genres/blues/blues.0000...,blues,./genres/blues/blues.00004.wav


In [2]:
# Analyze the data amount of each category
dfagg = df.groupby(['style'])[['filename']].count()
dfagg

,filename
style,
blues,100
classical,100
country,100
disco,100
hiphop,100
jazz,100
metal,100
pop,100
reggae,100


In [3]:
# check file amount by os.walk
import os
from os.path import join, getsize
for root, dirs, files in os.walk(path):
    if root.split('/')[-1] in dfagg.index:
        print(root, "consumes ", end="")
        print(sum(getsize(join(root, name)) for name in files), end="")
        print(" bytes in ", len(files), " non-directory files ")

./genres/blues consumes 132363200 bytes in  100  non-directory files 
./genres/classical consumes 132427620 bytes in  100  non-directory files 
./genres/country consumes 132409952 bytes in  100  non-directory files 
./genres/disco consumes 132391360 bytes in  100  non-directory files 
./genres/hiphop consumes 132698048 bytes in  100  non-directory files 
./genres/jazz consumes 132451056 bytes in  100  non-directory files 
./genres/metal consumes 132323760 bytes in  100  non-directory files 
./genres/pop consumes 132305200 bytes in  100  non-directory files 
./genres/reggae consumes 132328980 bytes in  100  non-directory files 
./genres/rock consumes 132406516 bytes in  100  non-directory files 


In [4]:
import librosa
print('Your current version', librosa.__version__)
!pip install librosa==0

Your current version 0.8.1


ERROR: Could not find a version that satisfies the requirement librosa==0 (from versions: 0.2.0.linux-x86_64, 0.2.1.linux-x86_64, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0rc2, 0.4.0, 0.4.1rc0, 0.4.1, 0.4.2, 0.4.3rc0, 0.4.3, 0.5.0rc0, 0.5.0, 0.5.1, 0.6.0rc0, 0.6.0rc1, 0.6.0, 0.6.1rc0, 0.6.1, 0.6.2, 0.6.3, 0.7.0rc1, 0.7.0, 0.7.1, 0.7.2, 0.8.0, 0.8.1rc1, 0.8.1rc2, 0.8.1)
ERROR: No matching distribution found for librosa==0


In [5]:
# dataFrame 
cols = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']
for m in range(1, 21):
    cols.append('mfcc' + str(m))
cols.append('style')
print(cols)

['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'style']


In [12]:
# Getting features from audio files
import librosa
import numpy as np
from tqdm import tqdm
data = pd.DataFrame()
for songname in tqdm(df.filename.to_list()[:]):
    label = df[df.filename==songname]['style'].tolist()[0]
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse =  librosa.feature.rms(y=y)[0]
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    row=np.hstack(
                  ( songname.split('/')[-1],
                    np.mean(chroma_stft),
                    np.mean(rmse),
                    np.mean(spec_cent),
                    np.mean(spec_bw),
                    np.mean(rolloff),
                    np.mean(zcr),                
                   (np.mean(mfcc, axis=1)),
                    label
                  ) 
                 )
    #print(row)
    data = data.append(pd.DataFrame(row.reshape(1,-1)))
data.columns = cols    

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:44<00:00,  1.91it/s]


In [14]:
data[-3:]

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,style
0,rock.00097.wav,0.43210346,0.081616685,2077.190360945626,1926.9896775727282,4030.767293168295,0.12182428224167957,-125.03131,115.19498,-47.993507,...,10.481961,-17.951916,1.2111135,-11.534863,1.844774,-12.847901,3.4474251,-12.594178,-2.1070023,rock
0,rock.00098.wav,0.36234903,0.08388779,1398.6723575172969,1818.1484689191711,3014.7401036123742,0.04873137577399381,-224.9723,123.65689,-9.754534,...,7.683082,-10.071786,-0.25509754,-5.276486,-2.8162887,-4.4164376,1.5582654,-5.043121,-3.5855958,rock
0,rock.00099.wav,0.35819513,0.05446073,1609.4429191769846,1797.0650975953747,3246.280370186726,0.07628978811919504,-235.18933,123.88854,-22.536184,...,10.066769,-15.0195875,3.1104681,-7.1277537,1.7845151,-7.0704045,0.023642737,-2.0220344,1.1585249,rock


In [15]:
# check size
data.shape

(1000, 28)

In [16]:
data.to_pickle('data.pickle')

In [21]:
############################
#For skip data preprocessing
############################
import pandas as pd
sample = pd.read_pickle('data.pickle')
sample[:3]

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,style
0,blues.00000.wav,0.34994322,0.13022463,1784.4204464946633,2002.6501916232635,3806.4853160373937,0.08306639113293343,-113.59675,121.5573,-19.158825,...,8.810669,-3.6673682,5.751691,-5.1627626,0.75094783,-1.6919376,-0.40995264,-2.3002088,1.2199285,blues
0,blues.00001.wav,0.3409832,0.095918424,1529.8353159300755,2038.6175789692213,3548.8202074363876,0.056044256724071206,-207.5568,124.00672,8.93056,...,5.376803,-2.2391195,4.2169633,-6.0122733,0.9361094,-0.71653724,0.29387605,-0.2874308,0.5315728,blues
0,blues.00002.wav,0.36360285,0.17557304,1552.4819582458863,1747.1659849613065,3040.514947755418,0.07630074799245357,-90.75439,140.4599,-29.109964,...,5.789265,-8.905224,-1.08372,-9.218358,2.455806,-7.726901,-1.815723,-3.4334342,-2.2268214,blues
